In [1]:
import pprint
import pandas as pd
import os
from pandas import ExcelWriter
from pandas import ExcelFile
import json

# WRITE DATA

In [2]:
import xlsxwriter as xl

sheets = ['inputs', 'outputs', 'start']



col_headers = {'inputs': ['Input_name', 'value', 'or filename', '', 'or MQTT params', 'Description'], 'outputs': ['Output_name', '', 'MQTT params', 'Description'], 'start': ['configs', 'Value', 'Description']} 

default_cell_values = {'inputs': [['','','url','',''],['','','topic','',''],['','','qos','','']], 'outputs': [['url','',''],['topic','',''],['qos','','']], 'start': [['','']] }



def generate_xlsx_instance_config(data, filepath):
        """Generate excel sheet for instance configuration
        
        Arguments:
            data {dict} -- Input data with list of config to be filled
            filepath {str} -- File path of the excel file to be created
        """
        if os.path.isfile(filepath):
            print("File already exists. Cannot overwrite file")
        else:

            with xl.Workbook(filepath) as workbook:

                row_header_format = workbook.add_format({'align': 'center', 'bold': True, 'valign': 'vcenter'})
                col_header_format = workbook.add_format({'align': 'center', 'bold': True, 'valign': 'vcenter'})    
                cell_format = workbook.add_format({'align':'center', 'valign': 'vcenter'})

                # Formatting information
                BORDER_SIZE = 2
                COL_HEADER_WIDTH = 30
                NORMAL_COL_WIDTH = 20
                SHORT_COL_WIDTH = 5

                row_header_format.set_border(BORDER_SIZE)
                col_header_format.set_border(BORDER_SIZE)
                cell_format.set_border(BORDER_SIZE)

                # Generate worksheets and format
                for worksheet_name in sheets:
                    sheet = workbook.add_worksheet(worksheet_name)

                    # Create column headers
                    col_header = col_headers[worksheet_name]
                    for col_num, col_value in enumerate(col_header):
                        sheet.write(0, col_num, col_value, col_header_format)

                        # Column formatting
                        sheet.set_column(col_num, col_num, NORMAL_COL_WIDTH)

                        # Widen first column
                        if col_num == 0:
                            sheet.set_column(col_num, col_num, COL_HEADER_WIDTH)
                        # Shorten empty column
                        if col_value == "":
                            sheet.set_column(col_num, col_num, SHORT_COL_WIDTH)


                    # Create row headers and format rows based on default inputs
                    row_header = data[worksheet_name]
                    cell_values = default_cell_values[worksheet_name]
                    for row_num, row_value in enumerate(row_header):

                        # Different formatting for start config sheet
                        if worksheet_name == "start":
                            sheet.write(row_num + 1, 0, row_value, cell_format)

                            for cell_row_num, cell_row in enumerate(cell_values):
                                for cell_col_num, cell_value in enumerate(cell_row):
                                    sheet.write(cell_row_num + row_num + 1, cell_col_num + 1, cell_value, cell_format)
                            # skip the rest of the code for start config
                            continue

                        # Merge every three rows for 
                        first_row = 1 + (row_num * 3)
                        last_row = first_row + 2
                        sheet.merge_range(first_row, 0, last_row, 0, row_value, row_header_format)

                        for cell_row_num, cell_row in enumerate(cell_values):
                            for cell_col_num, cell_value in enumerate(cell_row):
                                # Get col name and set formatting per row or per merged range
                                col_header_name = col_header[cell_col_num+1]
                                if "MQTT" in col_header_name or col_header_name == "":
                                    sheet.write(cell_row_num + first_row, cell_col_num + 1, cell_value, cell_format)
                                else:
                                    sheet.merge_range(first_row, cell_col_num + 1, last_row, cell_col_num + 1, "", cell_format)

In [186]:
data = {'inputs': ['ESS_Min_SoC', 'ESS_Max_SoC', 'SoC_Value', 'ESS_Capacity', 'ESS_Max_Charge_Power', 'ESS_Max_Discharge_Power', 'P_Grid_Max_Export_Power', 'P_PV', 'PV_Inv_Max_Power', 'P_Load'], 'outputs': ['P_Grid_Output', 'P_PV_Output', 'P_ESS_Output', 'SoC_ESS'], 'start': ['control_frequency', 'horizon_in_steps', 'dT_in_seconds', 'model_name', 'repetition', 'solver']}
filepath="../notebooks/instance-name.xlsx"

generate_xlsx_instance_config(data, filepath)

File already exists. Cannot overwrite file


# READ DATA

In [13]:
def read_data_from_xlsx_instance_config(filepath):
    
    if not os.path.isfile(filepath):
        print(f"Error: Excel file at {filepath} is missing")
        return

    # Read file
    excel_data = pd.read_excel(filepath, sheet_name=None)

    # Extract inputs sheet
    inputs = excel_data["inputs"]
    inputs.drop(labels=["Description", inputs.columns[3]], axis=1, inplace=True)
    inputs.fillna("empty_input_values", inplace=True)
    
    generic_input_mqtt = {}
    generic_input_dataset = {}
    input_fields = []
    
    # Extract data from input sheet and store as dict
    for row_num in range(0, len(inputs), 3):
        input_value_name = inputs.loc[row_num]["Input_name"]
        input_fields.append(input_value_name)
        
        input_values = inputs.loc[row_num][1:]

        for key, value in input_values.items():
            if value == "empty_input_values":
                continue

            if "MQTT params" in key:
                host = inputs.loc[row_num]["or MQTT params"]
                topic = inputs.loc[row_num+1]["or MQTT params"]
                qos = inputs.loc[row_num+2]["or MQTT params"]

                if input_value_name in generic_input_mqtt:
                    print(f"ERROR: Duplicate values: Please fill only one column for {input_value_name} in inputs sheet")
                    return
                
                                
                if host != "empty_input_values" and topic != "empty_input_values" and qos != "empty_input_values":
                    generic_input_mqtt[input_value_name] = {
                        "mqtt": {
                            "qos": qos,
                            "host": host,
                            "topic": topic
                        }
                    }
                else:
                    print(f"ERROR: MQTT params for {input_value_name} in inputs sheet is missing.")
                    return
                
                continue

            if "filename" in key:
                if os.path.isfile(value):
                    data_from_file = pd.read_excel(value, header=None)
                    
                    if input_value_name in generic_input_dataset:
                        print(f"ERROR: Duplicate values: Please fill only one column for {input_value_name}")
                        return
                        
                    generic_input_dataset[input_value_name] = data_from_file[data_from_file.columns[0]].tolist()
                else:
                    print(f"ERROR: Filename {value} provided for input {input_value_name} in inputs sheet is missing. Please check again")
                    return
                continue

            if input_value_name in generic_input_mqtt:
                    print(f"ERROR: Duplicate values: Please fill only one column for {input_value_name} in inputs sheet")
                    return
            
            if value.isnumeric():
                generic_input_mqtt[input_value_name] = value
            else:
                print(f"ERROR: Please enter numeric values for {input_value_name} in inputs")

    filled_inputs = set(generic_input_mqtt).union(set(generic_input_dataset))
    missing_inputs = set(input_fields).difference(filled_inputs)

    for input_name in missing_inputs:
        print(f"ERROR: {input_name} field in inputs sheet is missing")
        return
            
    # Extract outputs sheet
    outputs = excel_data["outputs"].drop(labels=["Description"], axis=1)
    outputs.drop(outputs.columns[1], axis=1, inplace=True)
    outputs.fillna("empty_input_values", inplace=True)
    
    generic_output_data = {}  
    output_fields = []
    
    # Extract data from output sheet and store as dict
    for row_num in range(0, len(outputs), 3):
        output_value_name = outputs.loc[row_num]["Output_name"]
        output_fields.append(output_value_name)

        host = outputs.loc[row_num]["MQTT params"]
        topic = outputs.loc[row_num+1]["MQTT params"]
        qos = outputs.loc[row_num+2]["MQTT params"]
        
        if host != "empty_input_values" and topic != "empty_input_values" and qos != "empty_input_values":
            generic_output_data[output_value_name] = {
                "mqtt": {
                    "qos": qos,
                    "host": host,
                    "topic": topic
                }
            }
        else:
            print(f"ERROR: MQTT params for {output_value_name} in inputs sheet is missing.")
            return

    missing_outputs = set(output_fields).difference(set(generic_output_data))
    for output_name in missing_outputs:
        print(f"ERROR: {output_name} field in outputs sheet is missing")
        return
        
    # Extract data from start sheet and store it as dict
    start_config = excel_data["start"].drop(labels=["Description"], axis=1)
    start_config.fillna("empty_input_values", inplace=True)
    
    generic_start_config_data = {}
    for i, row in start_config.iterrows():
        config_name = row["configs"]
        config_value = row["Value"]
        if config_value != "empty_input_values":
            if config_name == ""
            generic_start_config_data[config_name] = config_value
        else:
            print(f"ERROR: {config_name} field in start config sheet is missing")
            return


    # Store all data in a dict
    data_from_xlsx = {
        "inputs": {
            "dataset": generic_input_dataset,
            "mqtt": generic_input_mqtt
        },
        "outputs": {
            "generic": generic_output_data
        },
        "start": generic_start_config_data
    }
    
    return data_from_xlsx

In [14]:
filename="./gustavo.xlsx"
data_from_xlsx = read_data_from_xlsx_instance_config(filename)

pprint.pprint(data_from_xlsx, width=10)

AttributeError: 'float' object has no attribute 'isnumeric'